# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/Sam/Desktop/MY_UC_BERK/CHALLENGES/CHALLENGE_6/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,12.41,68,99,17.76,SH,1699064089
1,1,ancud,-41.8697,-73.8203,8.85,89,100,2.39,CL,1699064091
2,2,gondia,21.4500,80.2000,20.67,61,0,1.36,IN,1699064092
3,3,adamstown,-25.0660,-130.1015,22.89,76,100,5.15,PN,1699064092
4,4,nurlat,54.4277,50.8051,0.42,82,6,1.85,RU,1699064048


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
scatter_map = folium.Map(location=[0,0], zoom_start=2)

# Display the map

# Iterate through the DataFrame and add markers for each city
for index, row in city_data_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    lon = row["Lng"]
    humidity = row["Humidity"]
    
    # Create a circle marker with the size based on humidity
    marker = folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/25,  # Adjust the scale factor to control marker size
        popup=f"City: {city}<br>Humidity: {humidity}%",
        fill="blue",
        fill_opacity=0.5,
    ) 
    marker.add_to(scatter_map)

scatter_map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
perf_temp = city_data_df['Max Temp'] == 21
perf_humid = city_data_df['Humidity'] == 5
perf_cloudiness = city_data_df['Cloudiness'] ==0 
perf_wind_speed = city_data_df['Wind Speed'] > 0
perf_day = perf_temp & perf_humid & perf_cloudiness & perf_wind_speed

# Drop any rows with null values
city_data_df.drop(city_data_df[perf_day].index, inplace=True)


# Display sample data
len(city_data_df)

367

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,68,
1,ancud,CL,-41.8697,-73.8203,89,
2,gondia,IN,21.4500,80.2000,61,
4,nurlat,RU,54.4277,50.8051,82,
5,isafjordur,IS,66.0755,-23.1240,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "type": "accommodation", # Filter for hotel places
    "bias": "proximity",  # Bias the search based on proximity to the city's coordinates
    "apiKey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")
    
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["type"] = "accommodation"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


hotel_df
    

Starting hotel search
edinburgh of the seven seas - nearest hotel: Sea View
ancud - nearest hotel: El rincon
gondia - nearest hotel: The Grand Sita
nurlat - nearest hotel: Гостиница «Нурлат»
isafjordur - nearest hotel: Gamla Gistihusið
lebu - nearest hotel: Los filtros
grytviken - nearest hotel: Lookout Lodge
waitangi - nearest hotel: Travelers Rest
tamazulapam villa del progreso - nearest hotel: Balneario de Aguas Azufradas "Atonaltzin"
iqaluit - nearest hotel: Frobisher Inn
ponta delgada - nearest hotel: Guest House Bella Italia
iskateley - nearest hotel: Авантаж
pulandian - nearest hotel: 清泉宫温泉会馆
yellowknife - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
papatowai - nearest hotel: Cathedral Cave Cottage
freistadt - nearest hotel: Haus in der Höll
port-aux-francais - nearest hotel: Keravel
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
jamestown - nearest hotel: DoubleTree Jamestown
ushuaia - nearest hotel: Apart Hotel Aires del Beagl

tazovsky - nearest hotel: Тазовчанка
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
lensk - nearest hotel: No hotel found
severo-yeniseyskiy - nearest hotel: Актолик
keflavik - nearest hotel: Núpan Deluxe
yevlakh - nearest hotel: Vağzal Otel Mehmanxana
ubari - nearest hotel: فندق اوباري
vicuna mackenna - nearest hotel: Hospedaje San José
egilsstadir - nearest hotel: Sólbrekka 16 Guesthouse
grand falls-windsor - nearest hotel: Hotel Robin Hood
pacific grove - nearest hotel: Pacific Grove Inn
el calafate - nearest hotel: Posada Karut Josh
lichinga - nearest hotel: Residencial 2+1
mafeteng - nearest hotel: Mafeteng Hotel
klyuchi - nearest hotel: Туристическая гостиница
turbat - nearest hotel: Juddi Hotel
bodo - nearest hotel: Nordlysbuen Apartments
praia da vitoria - nearest hotel: Salles
selfoss - nearest hotel: Gesthús
mananjary - nearest hotel: Ideal Hotel Annexe
chifeng - nearest hotel: 心连心酒店
ola - nearest hotel: База отдыха "Геолог"
glace bay - nearest hotel: B

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,68,Sea View
1,ancud,CL,-41.8697,-73.8203,89,El rincon
2,gondia,IN,21.4500,80.2000,61,The Grand Sita
4,nurlat,RU,54.4277,50.8051,82,Гостиница «Нурлат»
5,isafjordur,IS,66.0755,-23.1240,90,Gamla Gistihusið
...,...,...,...,...,...,...
561,salym,RU,60.0625,71.4789,85,Каравани
563,chui,UY,-33.6971,-53.4616,82,Gaboto Chuy
564,puerto palomas,MX,31.7667,-107.6167,22,Motel Santa Cruz
566,ghadamis,LY,30.1337,9.5007,28,فندق عين الفارس


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
map_center = [hotel_df.iloc[0]["Lat"], hotel_df.iloc[0]["Lng"]]
my_map = folium.Map(location=map_center, zoom_start=2)

# Iterate through the DataFrame to add markers for each city and its nearest hotel
for index, row in hotel_df.iterrows():
    city_location = [row["Lat"], row["Lng"]]
    hotel_name = hotel_df.loc[index, "Hotel Name"]
    country = hotel_df.loc[index, "Country"]
    
    # Create a marker with a popup containing information about the city and its nearest hotel
    popup_content = f"City: {row['City']}<br>Country: {country}<br>Nearest Hotel: {hotel_name}"
    
    marker = folium.CircleMarker(location=city_location, radius=4, popup=popup_content)
    
    # Add the marker to the map
    marker.add_to(my_map)

# Display the map
my_map